# Predicting Wine Quality Using Linear Regression

## Summary

## Introduction

## Methods

### Loading Data
From UCI Machine Learning Repository: <https://doi.org/10.24432/C56S3T>

In [8]:
# Import packages
library(tidyverse)

In [22]:
wine <- read_delim("data/winequality-red.csv", delim = ";")
new_names <- c("fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar", "chlorides", "free_sulfur_dioxide", 
              "total_sulfur_dioxide", "density", "pH", "sulphates", "alcohol", "quality")
colnames(wine) <- new_names
head(wine)

Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
dbl (12): fixed acidity, volatile acidity, citric acid, residual sugar, chlo...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5


### EDA 

In [27]:
# Check for NA values
sum(is.na(wine))

[1] 0

In [35]:
# Summary Statistics
summary(wine)

 fixed_acidity   volatile_acidity  citric_acid    residual_sugar  
 Min.   : 4.60   Min.   :0.1200   Min.   :0.000   Min.   : 0.900  
 1st Qu.: 7.10   1st Qu.:0.3900   1st Qu.:0.090   1st Qu.: 1.900  
 Median : 7.90   Median :0.5200   Median :0.260   Median : 2.200  
 Mean   : 8.32   Mean   :0.5278   Mean   :0.271   Mean   : 2.539  
 3rd Qu.: 9.20   3rd Qu.:0.6400   3rd Qu.:0.420   3rd Qu.: 2.600  
 Max.   :15.90   Max.   :1.5800   Max.   :1.000   Max.   :15.500  
   chlorides       free_sulfur_dioxide total_sulfur_dioxide    density      
 Min.   :0.01200   Min.   : 1.00       Min.   :  6.00       Min.   :0.9901  
 1st Qu.:0.07000   1st Qu.: 7.00       1st Qu.: 22.00       1st Qu.:0.9956  
 Median :0.07900   Median :14.00       Median : 38.00       Median :0.9968  
 Mean   :0.08747   Mean   :15.87       Mean   : 46.47       Mean   :0.9967  
 3rd Qu.:0.09000   3rd Qu.:21.00       3rd Qu.: 62.00       3rd Qu.:0.9978  
 Max.   :0.61100   Max.   :72.00       Max.   :289.00       Max.   :1

In [42]:
# Means for each level of response variable "quality"
wine$quality <- as.factor(wine$quality)
response_means <- wine %>% 
    group_by(quality) %>% 
    summarise_all(mean)
response_means

quality,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,8.360000,0.8845000,0.1710000,2.635000,0.12250000,11.00000,24.90000,0.9974640,3.398000,0.5700000,9.955000
4,7.779245,0.6939623,0.1741509,2.694340,0.09067925,12.26415,36.24528,0.9965425,3.381509,0.5964151,10.265094
5,8.167254,0.5770411,0.2436858,2.528855,0.09273568,16.98385,56.51395,0.9971036,3.304949,0.6209692,9.899706
6,8.347179,0.4974843,0.2738245,2.477194,0.08495611,15.71160,40.86991,0.9966151,3.318072,0.6753292,10.629519
7,8.872362,0.4039196,0.3751759,2.720603,0.07658794,14.04523,35.02010,0.9961043,3.290754,0.7412563,11.465913
8,8.566667,0.4233333,0.3911111,2.577778,0.06844444,13.27778,33.44444,0.9952122,3.267222,0.7677778,12.094444


### EDA Visualization

### Regression

### Regression Visualization

## Discussion

### Findings

### Impacts and Future Questions

## References